# Dashboard
Das Dashboard zeigt alle erstellten Visualisierungen einschließlich einer kurzen Beschreibung auf einen Blick (https://plot.ly/python/create-online-dashboard/).

In [2]:
# zu implementieren